In [1]:
# --- Imports ---
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
import torch
import json
import os


c:\Users\LT\.conda\envs\llmpractice\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
pdf_path = "C:/wajahat/personal/learning/AI_agents_service_providers/stove.pdf"

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Loaded {len(documents)} pages")
print(documents[0].page_content[:500])

Loaded 7 pages
THE SCIENCE 
BEHIND HEAT
Large Electric Stove with 
Flame Effect 
Instruction Manual


In [9]:
print(documents[5].page_content[:50])

In [14]:
import fitz  # PyMuPDF
from tqdm.auto import tqdm
from typing import List, Dict

def extract_text_from_pdf(text: str) -> str:

    cleaned_text = text.replace('\n', " ").strip()    

    return cleaned_text

def open_read_pdf(pdf_path: str) -> List[Dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = extract_text_from_pdf(text=text)
        pages_and_text.append({ "page_num": page_num,
                               "page_char_count": len(text),
                               "page_word_count": len(text.split(" ")),
                               "page_sentences_count": len(text.split(". ")),
                               "page_tokens_count": len(text) / 4,
                                 "text": text })
        
    return pages_and_text

pages_and_text = open_read_pdf(pdf_path=pdf_path)
pages_and_text[:5]

7it [00:00, 223.08it/s]


[{'page_num': 0,
  'page_char_count': 84,
  'page_word_count': 15,
  'page_sentences_count': 1,
  'page_tokens_count': 21.0,
  'text': 'THE SCIENCE  BEHIND HEAT Large Electric Stove with  Flame Effect  Instruction Manual'},
 {'page_num': 1,
  'page_char_count': 2009,
  'page_word_count': 357,
  'page_sentences_count': 19,
  'page_tokens_count': 502.25,
  'text': 'THE SCIENCE  BEHIND HEAT Getting Started Remove the appliance from the box.  Remove any packaging from the appliance.  Place the packaging inside the box and either store or dispose of safely. In the Box Large 2000W stove fire   Instruction manual Features Variable flame brightness control   Adjustable thermostat  Fan heater with two heat settings  Thermal safety cut-out  1000/2000W heat setting switches  Free standing Safety Instructions Do not use the appliance until it is fully assembled as described in this manual.  When using this electrical appliance, basic safety precautions should be followed to reduce the  risk of fir

In [2]:
# === CONFIG ===
pdf_path = "C:/wajahat/personal/learning/AI_agents_service_providers/stove.pdf"          # Path to your PDF
chroma_dir = "C:/wajahat/personal/learning/AI_agents_service_providers/chroma_db/stove"                 # Folder to persist embeddings
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

# === LOAD & SPLIT DOCUMENT ===
loader = PyPDFLoader(pdf_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)

# === EMBEDDINGS ===
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

# === VECTOR STORE (Chroma) ===
vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory=chroma_dir)
vectorstore.persist()

print(f"✅ Stored {len(docs)} chunks into Chroma DB at: {chroma_dir}")


C:\Users\LT\AppData\Local\Temp\ipykernel_24016\2091549473.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)


✅ Stored 15 chunks into Chroma DB at: C:/wajahat/personal/learning/AI_agents_service_providers/chroma_db/stove


C:\Users\LT\AppData\Local\Temp\ipykernel_24016\2091549473.py:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
import torch
import ollama

#1. Create a quantization config
quantization_config = BitsAndBytesConfig( load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

attn_implementation = "flash_attention_2" if is_flash_attn_2_available() else "sdpa"

#2. Model Id
# model_id = "openai-community/gpt2"
model_id = "Qwen/Qwen2.5-1.5B-Instruct"
# model_path = "ollama/llama2"
# client = ollama.Client()
# model_path = llama2

#3. Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

#4. Load the model
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                               quantization_config=quantization_config,
                                               attn_implementation=attn_implementation,
                                               low_cpu_mem_usage=False,)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# === CONFIG ===
model_path = "ibm-granite/granite-4.0-micro"  # or "mistralai/Mistral-7B-v0.1"

# === LOAD EMBEDDINGS & VECTORSTORE ===
# embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
# vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)

# === LOAD MODEL ===
# device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

generator = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    # device=device,
    max_new_tokens=512,
    temperature=0.1,
)
llm = HuggingFacePipeline(pipeline=generator)

# # === CREATE RETRIEVAL CHAIN ===
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
#     return_source_documents=False,
# )

print("✅ RAG pipeline ready!")


Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.72s/it]


NameError: name 'device' is not defined

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

generator = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    # device=device,
    max_new_tokens=512,
    temperature=0.1,
)

llm = HuggingFacePipeline(pipeline=generator)

# === CREATE RETRIEVAL CHAIN ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=False,
)

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [ ]:
from sentence_transformers import SentenceTransformer, util

# === CONFIG ===

model = f"{model_path.replace('/', '_')}"
quries = "stiching"
queries_path = f"{quries}.json"       # JSON file containing { "questions": [...], "answers": [...] }
output_file = f"{model_path}_{quries}_responses.txt"

# Example format for queries.json:
# {
#   "questions": [
#       "What is the main objective of the project?",
#       "How long is the duration?"
#   ],
#   "answers": [
#       "The main objective is to monitor and analyze energy usage patterns.",
#       "The duration is one year."
#   ]
# }

# === LOAD QUESTIONS & ANSWERS ===
with open(queries_path, "r", encoding="utf-8") as f:
    data = json.load(f)

questions = data["questions"]
ground_truths = data["answers"]

# === EMBEDDING MODEL FOR SIMILARITY ===
sim_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# === RUN INFERENCE LOOP ===
results = []
for i, (q, ref) in enumerate(zip(questions, ground_truths)):
    print(f"\n[{i+1}/{len(questions)}] Question: {q}")
    model_answer = qa_chain.run(q)

    # Compute similarity
    emb_ref = sim_model.encode(ref, convert_to_tensor=True)
    emb_ans = sim_model.encode(model_answer, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(emb_ref, emb_ans).item()

    # Save to list
    results.append({
        "question": q,
        "reference_answer": ref,
        "model_answer": model_answer,
        "similarity": similarity
    })

# === SAVE TO TXT ===
with open(output_file, "w", encoding="utf-8") as f:
    for r in results:
        f.write(f"Question: {r['question']}\n")
        f.write(f"Reference: {r['reference_answer']}\n")
        f.write(f"Model: {r['model_answer']}\n")
        f.write(f"Similarity: {r['similarity']:.4f}\n")
        f.write("="*80 + "\n")

print(f"\n✅ Done! Responses and similarities saved in {output_file}")
